In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.utils as ku 
import pickle
import logging

from generators import PasswordBatchGenerator
from data_preparation import construct_subsequences, ff_labels, rnn_labels, add_spaces, transform_password, pad_and_tokenize
from predict_utils import predict_letter, predict_next_letter, predict_password

from keras.preprocessing.text import Tokenizer

print(tf.keras.__version__)
tf.test.gpu_device_name()

2.11.0


2023-02-26 11:53:36.629259: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-26 11:53:36.629743: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-26 11:53:36.630024: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-26 11:53:36.630817: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-26 11:53:36.630848: I tensorflow/core/co

'/device:GPU:0'

In [26]:
try:
    with open('data.pkl', 'rb') as file:
        df = pickle.load(file)
except:
    # Load the Password list into Memory
    url = "https://raw.githubusercontent.com/danielmiessler/SecLists/master/Passwords/Common-Credentials/10-million-password-list-top-1000000.txt"

    df_in = pd.read_csv(url, header=None)

    df = transform_password(df_in)
    with open('data.pkl', 'wb') as file:
        # Cache Data
        pickle.dump(df, file)

In [27]:
limit = 10_000 # len(df)
sequences = construct_subsequences(df, limit=limit)

In [37]:
letter_case = np.array( [ [np.int32(l.isupper()+1) for l in letters] for letters in sequences] )

/tmp/ipykernel_27523/168904925.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  letter_case = np.array( [ [np.int32(l.isupper()+1) for l in letters] for letters in sequences] )


In [29]:
# We define two different tokenizer, one for capital letters and one for letters

tokenizer_letter  = Tokenizer(filters=None, char_level=True, lower=True)
tokenizer_letter.fit_on_texts(sequences)


In [30]:
#tokenizer_case.texts_to_sequences(letter_case)

max_length = max([len(x) for x in sequences])

# Translation from Word to Token and back
word2idx = tokenizer_letter.word_index
idx2word = tokenizer_letter.index_word
vocab_size = len(word2idx) + 1

predictors_letter, label_letter = rnn_labels(pad_and_tokenize(sequences, tokenizer_letter, max_length), vocab_size)

In [38]:
# For Binaries we do not need a tokenizer, we just pad
predictors_case = np.array([ np.pad(x[0:-1], (max_length-len(x[0:-1])-1, 0), constant_values=0) for x in letter_case])
label_case = np.array([x[-1] for x in letter_case])

In [42]:
tf.keras.layers.Concatenate(axis=1)([predictors_letter, predictors_case])

<tf.Tensor: shape=(56508, 32), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]], dtype=int32)>

In [ ]:
# Init generator
# We are not using a generator for now
# batch_size= 10_000
# training_batch = PasswordBatchGenerator(predictors, np.array(labels), batch_size)

: 

In [ ]:
try:
  with open(f'/app/models/lstm_model_multivar_{str(limit)}.pkl', 'rb') as file:
      lstm_model = pickle.load(file)
      logging.info(lstm_model.summary())
except:
  lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 10, input_length=max_length-1), #Turns positive integers (indexes) into dense vectors of fixed size.
    tf.keras.layers.Masking(mask_value=0),
    tf.keras.layers.LSTM(100), # skips masked timesteps
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(vocab_size+1, activation='softmax')
  ])
  lstm_model.compile(
                #optimizer='rmsprop',
                loss='categorical_crossentropy',
                optimizer='adam')

  lstm_model.fit( training_batch , epochs=100, verbose=1)

  # cache the model
  with open(f'/app/models/lstm_model_{str(limit)}.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(lstm_model, file)


: 

In [ ]:
from keras.layers import Input, Dense, Dropout, LSTM

# letter_inputs = tf.keras.layers.Embedding(vocab_size, 10, input_length=max_length-1) #
letter_inputs = tf.keras.layers.Input(shape=max_length-1) # 7 past steps and variables
case_inputs = tf.keras.layers.Input(shape=max_length-1) # 7 past steps and variables

inputs = tf.keras.layers.concatenate([letter_inputs, case_inputs])

# m = tf.keras.layers.Embedding(vocab_size, 10, input_length=max_length-1)(m)
m = tf.keras.layers.Masking(mask_value=0)(inputs)
m = tf.keras.layers.LSTM(100)(m)
m = tf.keras.layers.Dropout(0.1)(m)
outputA = tf.keras.layers.Dense(vocab_size, activation='softmax')(m)
outputB = tf.keras.layers.Dense(1, activation='softmax')(m)

m = tf.keras.Model(inputs=[inputs], outputs=[outputA, outputB])
m.compile(optimizer='adam', loss='categorical_crossentropy')
# m.fit(training_batch,  epochs=100, verbose=1 )


: 

In [ ]:
# https://stackoverflow.com/questions/66845924/multi-input-multi-output-model-with-keras-functional-api

import tensorflow.keras as keras

(xtrain, ytrain), (xtest, _) = keras.datasets.mnist.load_data()

xtrain = xtrain[:10000] # both input sample should be same number 
ytrain = ytrain[:10000] # both input sample should be same number

y_out_a = keras.utils.to_categorical(ytrain, num_classes=10)
y_out_b = keras.utils.to_categorical((ytrain % 2 == 0).astype(int), num_classes=2)
y_out_c = tf.square(tf.cast(ytrain, tf.float32))

print(xtrain.shape, xtest.shape) 
print(y_out_a.shape, y_out_b.shape, y_out_c.shape)
# (10000, 28, 28) (10000, 28, 28)
# (10000, 10) (10000, 2) (10000,)

# Next, we need to modify some parts of the above model to take multi-input. And next if you now plot, you will see the new graph.

input0 = keras.Input(shape=(28, 28, 1), name="img2")
input1 = keras.Input(shape=(28, 28, 1), name="img1")
concate_input = keras.layers.Concatenate()([input0, input1])

x = keras.layers.Conv2D(16, 3, activation="relu")(concate_input)
x = keras.layers.Conv2D(32, 3, activation="relu")(x)
x = keras.layers.MaxPooling2D(3)(x)
x = keras.layers.Conv2D(32, 3, activation="relu")(x)
x = keras.layers.Conv2D(16, 3, activation="relu")(x)
x = keras.layers.GlobalMaxPooling2D()(x)

out_a = keras.layers.Dense(10, activation='softmax', name='10cls')(x)
out_b = keras.layers.Dense(2, activation='softmax', name='2cls')(x)
out_c = keras.layers.Dense(1, activation='linear', name='1rg')(x)


# multi-input , multi-output
encoder = keras.Model( inputs = [input0, input1], 
                       outputs = [out_a, out_b, out_c], name="encoder")

keras.utils.plot_model(
    encoder
)
encoder.compile(
    loss = {
        "10cls": tf.keras.losses.CategoricalCrossentropy(),
        "2cls": tf.keras.losses.CategoricalCrossentropy(),
        "1rg": tf.keras.losses.MeanSquaredError()
    },

    metrics = {
        "10cls": 'accuracy',
        "2cls": 'accuracy',
        "1rg": 'mse'
    },

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
)
encoder.fit([xtrain, xtest], [y_out_a, y_out_b, y_out_c], 
             epochs=30, batch_size = 256, verbose=2)



: 

In [ ]:
# https://stackoverflow.com/questions/66845924/multi-input-multi-output-model-with-keras-functional-api

import tensorflow.keras as keras

(xtrain, ytrain), (xtest, _) = keras.datasets.mnist.load_data()

xtrain = xtrain[:10000] # both input sample should be same number 
ytrain = ytrain[:10000] # both input sample should be same number

y_out_a = keras.utils.to_categorical(ytrain, num_classes=10)
y_out_b = keras.utils.to_categorical((ytrain % 2 == 0).astype(int), num_classes=2)
y_out_c = tf.square(tf.cast(ytrain, tf.float32))

print(xtrain.shape, xtest.shape) 
print(y_out_a.shape, y_out_b.shape, y_out_c.shape)
# (10000, 28, 28) (10000, 28, 28)
# (10000, 10) (10000, 2) (10000,)

# Next, we need to modify some parts of the above model to take multi-input. And next if you now plot, you will see the new graph.

input0 = keras.Input(shape=(28, 28, 1), name="img2")
input1 = keras.Input(shape=(28, 28, 1), name="img1")
concate_input = keras.layers.Concatenate()([input0, input1])

x = keras.layers.Conv2D(16, 3, activation="relu")(concate_input)
x = keras.layers.Conv2D(32, 3, activation="relu")(x)
x = keras.layers.MaxPooling2D(3)(x)
x = keras.layers.Conv2D(32, 3, activation="relu")(x)
x = keras.layers.Conv2D(16, 3, activation="relu")(x)
x = keras.layers.GlobalMaxPooling2D()(x)

out_a = keras.layers.Dense(10, activation='softmax', name='10cls')(x)
out_b = keras.layers.Dense(2, activation='softmax', name='2cls')(x)
out_c = keras.layers.Dense(1, activation='linear', name='1rg')(x)


# multi-input , multi-output
encoder = keras.Model( inputs = [input0, input1], 
                       outputs = [out_a, out_b, out_c], name="encoder")

keras.utils.plot_model(
    encoder
)
encoder.compile(
    loss = {
        "10cls": tf.keras.losses.CategoricalCrossentropy(),
        "2cls": tf.keras.losses.CategoricalCrossentropy(),
        "1rg": tf.keras.losses.MeanSquaredError()
    },

    metrics = {
        "10cls": 'accuracy',
        "2cls": 'accuracy',
        "1rg": 'mse'
    },

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
)
encoder.fit([xtrain, xtest], [y_out_a, y_out_b, y_out_c], 
             epochs=30, batch_size = 256, verbose=2)



: 

In [ ]:
keras.utils.plot_model(
    encoder
)

: 

In [ ]:
arr = np.array( predictors )

: 

In [ ]:
arr.shape

: 

In [ ]:
import pickle

# pickle model
with open('/root/model.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(lstm_model, file)

: 

In [ ]:
predict_next_letter("lov", ff_model, tokenizer, True)
predict_next_letter("lov", rnn_model, tokenizer, True)
predict_next_letter("lov", lstm_model, tokenizer, True)

: 

In [ ]:
print( predict_password("lov", ff_model, tokenizer) )
print( predict_password("lov", rnn_model, tokenizer) )
print( predict_password("lov", lstm_model, tokenizer) )

: 

In [ ]:
#https://jhui.github.io/2017/03/15/RNN-LSTM-GRU/
#https://www.simplilearn.com/tutorials/deep-learning-tutorial/rnn
#https://www.kaggle.com/code/namansood/nlp-guide-next-word-prediction-and-deep-learning
#https://stackoverflow.com/questions/43341374/tensorflow-dynamic-rnn-lstm-how-to-format-input
#https://r2rt.com/recurrent-neural-networks-in-tensorflow-i.html
#https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275
#https://ai.stackexchange.com/questions/18198/what-is-the-difference-between-lstm-and-rnn#:~:text=The%20main%20difference%20between%20an%20LSTM%20unit%20and,better%20the%20flow%20of%20information%20through%20the%20unit.

: 